---------------------
IMPORTANDO AS BIBLIOTECAS
---------------------

In [2]:
import getpass
import pandas as pd
import numpy as np
import mysql.connector
import folium
import webbrowser

---------------------
IMPORTANDO AS TABELAS EXTERNAS
---------------------

In [3]:
dia = '22'
mes = '03'
ano = '2021'
url = f"""https://s3-sa-east-1.amazonaws.com/ckan.saude.gov.br/SRAG/2021/INFLUD21-{dia}-{mes}-{ano}.csv"""


In [4]:
data=pd.read_csv(str(url), sep=';', index_col=False)


c:\users\f1693478\appdata\local\programs\python\python37-32\lib\site-packages\IPython\core\interactiveshell.py:3166: DtypeWarning: Columns (15,18,60,62,63,64,67,92,94,106,108,115,117,118,119,123,144) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
data.head(3)

,DT_NOTIFIC,SEM_NOT,DT_SIN_PRI,SEM_PRI,SG_UF_NOT,ID_REGIONA,CO_REGIONA,ID_MUNICIP,CO_MUN_NOT,ID_UNIDADE,...,DS_AN_OUT,TP_AM_SOR,SOR_OUT,DT_CO_SOR,TP_SOR,OUT_SOR,DT_RES,RES_IGG,RES_IGM,RES_IGA
0,04/01/2021,1,03/01/2021,1,SP,GVE XVII CAMPINAS,1342.0,CAMPINAS,350950,HOSPITAL E MATERNIDADE CELSO PIERRO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,04/01/2021,1,03/01/2021,1,SP,GVE XXVII SAO JOSE DOS CAMPOS,1351.0,SAO JOSE DOS CAMPOS,354990,DR RUBENS SAVASTANO HOSPITAL REGIONAL DE SAO J...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,04/01/2021,1,03/01/2021,1,MG,BELO HORIZONTE,1449.0,MATEUS LEME,314070,MATEUS LEME UPA 24 HORAS TIAGO CARDOSO SANTOS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# TABELA DE OCUPAÇÃO CBO - MIN. TRABALHO
cbo = './CBO2002 - Ocupacao.csv'
cbo = pd.read_csv(cbo, sep=';', index_col=False, encoding='latin1')
cbo.head(3)

,CODIGO,TITULO
0,10105,Oficial general da aeronáutica
1,10110,Oficial general do exército
2,10115,Oficial general da marinha


In [10]:
def char_len(x, fixed_n):
        '''set string x to fixed_n character, prepend with 'xxx' if short'''
        if len(x) > fixed_n:
            return x[:fixed_n]
        elif len(x) < fixed_n:
            return 'x' * (fixed_n - len(x)) + x
        return x

In [11]:
# TABELA DE MUNICIPIOS DO IBGE
mun_ibge = './RELATORIO_DTB_BRASIL_MUNICIPIO.xls'
df = pd.read_excel(mun_ibge)
df['Código Município Completo']= df['Código Município Completo'].map(str)
df['Código Município Completo'] = df['Código Município Completo'].apply(lambda x: char_len(x, 6))
df_mun_ibge = df[['Código Município Completo','Nome_Município', 'Nome_UF']]
df_mun_ibge.columns = ['COD_MUN', 'NM_MUN', 'Nome_UF']
df_mun_ibge.head(3)


,COD_MUN,NM_MUN,Nome_UF
0,110001,Alta Floresta D'Oeste,Rondônia
1,110037,Alto Alegre dos Parecis,Rondônia
2,110040,Alto Paraíso,Rondônia


In [12]:
uf = './estados.xlsx'
df_uf = pd.read_excel(uf)
df_uf_ibge = df_uf[['Nome_UF', 'Sigla']]
# df_uf_ibge

---------------------
CONEXÃO COM O SERVIDOR MYSQL
---------------------

In [13]:
pword=getpass.getpass()

········


In [14]:
cnx = mysql.connector.connect(user='root', password=pword,
                              host='127.0.0.1',
                              database='srag')
cursor = cnx.cursor()

---------------------
CARGA DE TABELAS NO MYSQL
---------------------

In [16]:
#CARGA TABELA OCUPACAO
ocupa = cbo[['CODIGO', 'TITULO']]
ocupa = ocupa.reset_index(drop=True)
ocupa = ocupa.drop_duplicates()
df_ocupa = ocupa.replace(to_replace='None', value=np.nan).dropna()
df_ocupa = df_ocupa.replace(to_replace='XXX', value=0)
df_ocupa=df_ocupa.reset_index(drop=True)
for row in df_ocupa.itertuples():
    sql = "INSERT INTO ocupacao (cbo, descricao) VALUES (%s, %s)"
    val = (int(row.CODIGO), str(row.TITULO))
    cursor.execute(sql, val)
cnx.commit()
df_opr= pd.read_sql("SELECT * from ocupacao", cnx)
df_opr.head(3)

,cbo,descricao
0,0,Não informado
1,10105,Oficial general da aeronáutica
2,10110,Oficial general do exército


In [ ]:
 #CARGA TABELA ESTADO,
uf = df_uf_ibge[['Nome_UF', 'Sigla']]
uf
for row in uf.itertuples():
    sql = "INSERT INTO estado (sigla, descricao) VALUES (%s, %s)"
    val = ((str(row.Sigla), str(row.Nome_UF)))
    try:
        cursor.execute(sql, val)
    except:
        pass
cnx.commit()
df_uf= pd.read_sql("SELECT * from estado", cnx)
df_uf.head(3)

In [17]:
#CARGA TABELA MUNICIPIO
mun = df_mun_ibge[['COD_MUN','NM_MUN', 'Nome_UF']]
mun = pd.merge(df_uf_ibge, df_mun_ibge, on="Nome_UF")
for row in mun.itertuples():
    sql = "INSERT INTO municipio (codigo, descricao, Estado_sigla) VALUES (%s, %s, %s)"
    val = (int(row.COD_MUN), (row.NM_MUN), (row.Sigla))
    try:
        cursor.execute(sql, val)
    except:
        pass
cnx.commit()
df_mun= pd.read_sql("SELECT * from municipio", cnx)
df_mun.head(3)

,codigo,descricao,Estado_sigla
0,0,Não informado,XX
1,110001,Alta Floresta D'Oeste,RO
2,110002,Ariquemes,RO


In [22]:
# CARGA TABELA FATORES RISCOS
df_fator = pd.DataFrame({'idFator':[1,2,3,4,5,6,7,8,9,10,11,12],
                   'desc':['PUERPERA','CARDIOPATI','HEMATOLOGI','SIND_DOWN',
                           'HEPATICA','ASMA','DIABETES','NEUROLOGIC','PNEUMOPATI',
                          'IMUNODEPRE', 'RENAL', 'OBESIDADE']})
for row in df_fator.itertuples():
    sql = "INSERT INTO FatoresRiscos (idFatoresRiscos, descricao) VALUES (%s, %s)"
    val = (int(row.idFator), str(row.desc))
    try:
        cursor.execute(sql, val)
    except:
        pass
cnx.commit()
df_fator= pd.read_sql("SELECT * from FatoresRiscos", cnx)
df_fator.head(3)

,idFatoresRiscos,descricao
0,1,PUERPERA
1,2,CARDIOPATI
2,3,HEMATOLOGI


In [19]:
# CARGA TABELA PESSOA
pessoa = data[['CS_SEXO', 'CS_GESTANT','CS_RACA', 'CS_ESCOL_N', 'PAC_COCBO','PAC_DSCBO', 'ID_MUNICIP','CO_MUN_RES', 'DT_NASC', 'NU_IDADE_N']]
df_pessoa = pessoa.astype(object).where(pd.notnull(pessoa), None)
df_pessoa = df_pessoa.replace(to_replace='XXX', value=0)
df = df_pessoa.fillna(0)

# AJUSTES NA TABELA PESSOA - REGISTROS INCORRETOS DE MUNICIPIOS E CBO
df.loc[df.ID_MUNICIP == 'BRASILIA', 'CO_MUN_RES'] = 530010
df.loc[df.ID_MUNICIP == 'GOIANIA', 'CO_MUN_RES'] = 520870
df.loc[df.ID_MUNICIP == 'VALPARAISO DE GOIAS', 'CO_MUN_RES'] = 522185
df.loc[df.ID_MUNICIP == 'PORTO VELHO', 'CO_MUN_RES'] = 110020
df.loc[df.PAC_DSCBO == 'MONITOR DE TELEATENDIMENTO', 'PAC_COCBO'] = 422335
df.loc[df.PAC_DSCBO == 'AGENCIADOR DE PROPAGANDA', 'PAC_COCBO'] = 253140
df.loc[df.PAC_DSCBO == 'LOCUTOR PUBLICITARIO DE RADIO E TELEVISAO', 'PAC_COCBO'] = 261715
df.loc[df.PAC_DSCBO == 'AUXILIAR TECNICO EM PATOLOGIA CLINICA', 'PAC_COCBO'] = 324205

for row in df.itertuples():
    print(row)
    data_nasc = None
    try:
        from datetime import datetime
        data_nasc = datetime.strptime(row.DT_NASC, '%d/%m/%Y').date()
    except:
        data_nasc = None    
        
    sql = "INSERT INTO Pessoa (id_pessoa, sexo, gestante, raca, escolaridade, Ocupacao_cbo, Municipio_codigo, data_nascimento, nu_idade) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
    val = ((row.Index)+1, str(row.CS_SEXO), (row.CS_GESTANT), (row.CS_RACA), (row.CS_ESCOL_N), (row.PAC_COCBO), (row.CO_MUN_RES), (data_nasc), (row.NU_IDADE_N) )
    try:
        cursor.execute(sql, val)
    except:
        pass
cnx.commit()
df_pessoa= pd.read_sql("SELECT * from pessoa", cnx)
df_pessoa.head(3)

,id_pessoa,sexo,gestante,raca,escolaridade,Municipio_codigo,Ocupacao_cbo,data_nascimento,nu_idade
0,1,F,5,9,0,292740,0,1974-08-19,46
1,2,M,6,1,0,355030,0,2020-10-26,3
2,3,F,5,1,9,350950,0,1936-01-11,84


In [20]:
#CARGA TABELA UNIDADE
unidade = data[['ID_UNIDADE','CO_UNI_NOT', 'CO_MUN_NOT']]
unidade = unidade.drop_duplicates()
df_unidade =unidade.astype(object).where(pd.notnull(unidade), None)
for row in df_unidade.itertuples():  
    sql = "INSERT INTO unidade (id_unidade_saude, descricao, Municipio_codigo1) VALUES (%s, %s, %s)"
    val = (int(row.CO_UNI_NOT), str(row.ID_UNIDADE), int(row.CO_MUN_NOT))
    try:
        cursor.execute(sql, val)
    except:
        pass
cnx.commit()
df_pessoa= pd.read_sql("SELECT * from unidade", cnx)
df_pessoa.head(3)

,id_unidade_saude,descricao,Municipio_codigo1
0,19,POLICLINICA DR JAMACI DE MEDEIROS,260290
1,396,HOSPITAL DAS CLINICAS,261160
2,418,HOSPITAL AGAMENON MAGALHAES,261160


In [18]:
#CARGA TABELA REGISTRO
registro = data[['CO_UNI_NOT', 'DT_NOTIFIC', 'DT_ENCERRA', 'EVOLUCAO', 'CLASSI_FIN']]
df_registro =registro.astype(object).where(pd.notnull(registro), None)
# df_registro
for row in df_registro.itertuples():
    date_not= None
    date_enc= None
    try:
        from datetime import datetime
        date_not= datetime.strptime(row.DT_NOTIFIC, '%d/%m/%Y').date()
        date_enc= datetime.strptime(row.DT_ENCERRA, '%d/%m/%Y').date()
    except:
        pass    
    sql = "INSERT INTO registro (id_registro, Pessoa_id_pessoa, unidade_id_unidade_saude, data_notificacao,data_encerramento, evolucao)  VALUES (%s, %s, %s, %s, %s, %s)"
    val = ((row.Index)+1, (row.Index)+1, int(row.CO_UNI_NOT), (date_not), (date_enc), (row.EVOLUCAO))
    try:
        cursor.execute(sql, val)
    except:
        pass
cnx.commit()
df_registro= pd.read_sql("SELECT * from registro", cnx)
df_registro.head(3)

,id_registro,Pessoa_id_pessoa,unidade_id_unidade_saude,data_notificacao,data_encerramento,evolucao,classificacao
0,1,1,6595197,2021-01-05,2021-01-15,1.0,4.0
1,2,2,2082225,2021-01-04,2021-01-13,1.0,4.0
2,3,3,2082128,2021-01-04,2021-01-07,1.0,NaN


In [21]:
#CARGA TABELA INTERNACAO
internacao = data[['DT_INTERNA', 'CO_UNI_NOT', 'DT_EVOLUCA','DT_ENTUTI','DT_SAIDUTI']]
df_internacao =internacao.astype(object).where(pd.notnull(internacao), None)
id_internacao = 0
for row in df_internacao.itertuples():
    id_unidade_saude = None
    try:
#         APESAR DE ESTAR NO DICIONÁRIO DE DADOS O CAMPO CO_UN_INTE NÃO EXISTE
        id_unidade_saude=(row.CO_UN_INTE)
    except:
        id_unidade_saude=(row.CO_UNI_NOT)
    id_registro = (row.Index)+1
    
    if row.DT_INTERNA != None:
        data_ent = None
        data_ent_uti = None
        data_sai_uti = None
        try:
            from datetime import datetime
            data_ent = datetime.strptime(row.DT_INTERNA, '%d/%m/%Y').date()
            data_ent_uti = datetime.strptime(row.DT_ENTUTI, '%d/%m/%Y').date()
            data_sai_uti = datetime.strptime(row.DT_SAIDUTI, '%d/%m/%Y').date()

        except:
            pass

        sql = "INSERT INTO internacao (id_internacao, data_entrada_int, Registro_id_registro, unidade_id_unidade_saude, data_saida_uti, data_entrada_uti) VALUES (%s, %s, %s, %s, %s, %s)"
        id_internacao = id_internacao+1
        val = (id_internacao, data_ent, id_registro, id_unidade_saude, data_ent_uti, data_sai_uti)
        try:
            cursor.execute(sql, val)
        except:
            pass
cnx.commit()
df_internacao= pd.read_sql("SELECT * from internacao", cnx)
df_internacao.head(3)

,id_internacao,data_entrada_int,Registro_id_registro,unidade_id_unidade_saude,data_saida_uti,data_entrada_uti
0,1,2021-01-04,1,6595197,2021-01-04,None
1,2,2021-01-04,2,2082225,None,None
2,3,2021-01-03,3,2082128,None,None


TRANSPOSICAO DA TABELA FATORES RISCOS

In [25]:
#CARGA TABELA RELACIONAMENTO PESSOA x FATORES RISCOS
pessoa_has_fr = data[['PUERPERA', 'FATOR_RISC', 'CARDIOPATI', 'HEMATOLOGI', 'SIND_DOWN',
                'HEPATICA', 'ASMA', 'DIABETES', 'NEUROLOGIC', 'PNEUMOPATI',
                'IMUNODEPRE', 'RENAL', 'OBESIDADE']]
pessoa_has_fr["id"] = pessoa_has_fr.index + 1
df=pessoa_has_fr[pessoa_has_fr['FATOR_RISC']!='N']
del df['FATOR_RISC']
df.head(3)

c:\users\f1693478\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,PUERPERA,CARDIOPATI,HEMATOLOGI,SIND_DOWN,HEPATICA,ASMA,DIABETES,NEUROLOGIC,PNEUMOPATI,IMUNODEPRE,RENAL,OBESIDADE,id
0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1
2,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3
3,2.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,4


In [24]:
df.columns = [1,2,3,4,5,6,7,8,9,10,11,12,'id']
df_transposta =df.melt(id_vars=['id'], var_name='FATOR', value_name='VALOR')
df_transposta.loc[df_transposta.VALOR != 1, 'FATOR'] = np.nan
df_transposta = df_transposta.dropna()
df_transposta[df_transposta['id']==1]
for row in df_transposta.itertuples():
    sql = "INSERT INTO pessoa_has_fatoresriscos (Pessoa_id_pessoa, FatoresRiscos_idFatoresRiscos) VALUES (%s, %s)"
    val = (int(row.id),  int(row.FATOR) )
    try:
        cursor.execute(sql, val)
    except:
        pass
cnx.commit()

,id,FATOR,VALOR
213419,1,2,1.0
1280514,1,7,1.0


---------------
CRIANDO CONSULTAS NO WORKBENCH
---------------

1 - Quantos registros de doencas respiratorias X covid em qual intervalo.
2 - Numero de casos covid X obitos por estado, por nivel de instrucao, sexo , idade.
3 - Numero de casos covid (da consulta anterior)X comorbidades.
4 - Media de idade de casos confirmados por estado
5 - Media de registros de covid confirmados por estado X uti.

---------------
CRIANDO VIEWS NO WORKBENCH
---------------

In [ ]:
# create view srag.evolucao_casos_unidades as 
# select  b.descricao as nome_unidade, 
# 		c.descricao as municipio,
#         c.Estado_sigla as UF,
#         c.codigo as cod_mun,
#         (case when a.evolucao=1 then 'RECUPERADO'
# 			 when a.evolucao=2 then 'ÓBITO'
#              else 'N/A' end) as evolucao,
# 		count(Pessoa_id_pessoa) as total
# from registro as a
# left join srag.unidade as b
# 	on a.unidade_id_unidade_saude=b.id_unidade_saude
# left join srag.municipio as c
# 	on b.Municipio_codigo1=c.codigo
# group by a.unidade_id_unidade_saude, a.evolucao, b.descricao
# order by a.unidade_id_unidade_saude


---------------
CRIANDO PROCEDURES NO WORKBENCH
---------------

In [ ]:
# delimiter $$

# CREATE PROCEDURE casos_cidades (IN COD_US INT, OUT casos INT)
#        BEGIN
#          SELECT COUNT(*) INTO casos FROM srag.registro 
#          WHERE unidade_id_unidade_saude = COD_US;
#        END $$

# delimiter ;

# CALL casos_cidades(2082225, @casos); casos_cidades

---------------------
EXPLORANDO VIEWS EM FERRAMENTA GRÁFICA PARA VISÃO GERENCIAL
-----------------------


In [ ]:
# TABELA DE LATITUDES E LONGITUDE DE MUNÍCIPIOS
lat = './mun_ibge.csv'
df_lat = pd.read_csv(lat, sep=',', index_col=False, encoding='utf-8')
df_lat['codigo_ibge']= df_lat['codigo_ibge'].map(str)
df_lat['codigo_ibge'] = df_lat['codigo_ibge'].apply(lambda x: char_len(x, 6))
df_lat['codigo_ibge']=df_lat['codigo_ibge'].astype(int)

In [ ]:
# CONSULTANDO UMA VIEW
df_view= pd.read_sql("SELECT * from view_evol_casos_unidades", cnx)
df_view['cod_mun']=df_view['cod_mun'].astype(int)
df_view['estado']=df_view['UF']
df_view.head(5)

In [ ]:
tab_map = pd.merge(df_view, df_lat, left_on='cod_mun', right_on='codigo_ibge')
tab_map.head(1)

In [ ]:
coordenadas=[]
for lat,lng in zip(tab_map.latitude.values,tab_map.longitude.values):
    coordenadas.append([lat,lng])

In [ ]:
mapa = folium.Map(location=[-15.788497,-47.879873],zoom_start=4,tiles='Stamen Toner')
from folium import plugins
# Adicionando os registros no mapa de calor:
mapa.add_child(plugins.HeatMap(coordenadas,min_opacity=0.5, radius=10, gradient={0.2: 'blue', 0.4: 'lime', 0.8: 'red'}))        
mapa.save('srag_2021.html')
webbrowser.open("srag_2021.html")

In [ ]:
df_covid='./covid_dia.csv'
df=pd.read_csv(str(df_covid), sep=',', index_col=False)

In [ ]:
df.columns

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#criando uma figure, axes
fig, ax = plt.subplots()#criando o gráfico de barras 
sns.barplot(x=df.mes, y=df['total_notificado'], ax=ax, data=df)#otimizar espaço da figure
fig.tight_layout()

In [ ]:
cnx.close()